In [1]:
import os

import itertools
import pickle
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import math 

from torchtext import data, datasets
from pathlib import Path

import sys
sys.path.append('../')
%matplotlib inline

In [2]:
# Init cuda
device = "cuda:6" if torch.cuda.is_available() else "cpu"
idevice = 6 if torch.cuda.is_available() else -1
torch.cuda.set_device(idevice)
device, idevice

('cuda:6', 6)

In [3]:
TEXT = data.Field()
PATH = Path('./wikitext/')

In [4]:
batch_size = 256
sequence_length = 30
grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 10_000

In [5]:
from torchtext.datasets.language_modeling import LanguageModelingDataset

class WikiTextRu(LanguageModelingDataset):

    urls = ['http://files.deeppavlov.ai/datasets/wikitext_ru.zip']
    name = 'wikitext_ru'
    dirname = 'wikitext_ru'

    @classmethod
    def splits(cls, text_field, root='.data', 
               train='ru.wiki.train.txt', validation='ru.wiki.valid.txt', test='ru.wiki.test.txt',  **kwargs):

        return super().splits(text_field=text_field, root=root, 
                              train=train, validation=validation, test=test, **kwargs)

    @classmethod
    def iters(cls, batch_size=32, bptt_len=35, device=0, root='.data', vectors=None, **kwargs):
       
        TEXT = data.Field()
        train, val, test = cls.splits(TEXT, root=root, **kwargs)
        TEXT.build_vocab(train, vectors=vectors)

        return data.BPTTIterator.splits((train, val, test), 
                                        batch_size=batch_size, bptt_len=bptt_len, device=device)

In [6]:
from torchtext.vocab import Vectors

class RuFastText(Vectors):

    url_base = 'http://files.deeppavlov.ai/embeddings/ft_native_300_ru_wiki_lenta_nltk_word_tokenize/ft_native_300_ru_wiki_lenta_nltk_word_tokenize.vec'

    def __init__(self, **kwargs):
        url = self.url_base
        name = os.path.basename(url)
        super().__init__(name, url=url, **kwargs)


In [7]:
%%time
train_ds, valid_ds, test_ds = WikiTextRu.splits(TEXT, root=PATH)

CPU times: user 1min 22s, sys: 22.4 s, total: 1min 45s
Wall time: 1min 45s


In [8]:
ru_vectors = RuFastText()

In [9]:
%%time
TEXT.build_vocab(train_ds, min_freq=30, max_size=10_000, vectors=ru_vectors)

CPU times: user 2min 55s, sys: 18.7 s, total: 3min 14s
Wall time: 3min 13s


In [10]:
ntokens = len(TEXT.vocab)
ntokens

10002

In [11]:
%%time
train_loader, val_loader, test_loader = data.BPTTIterator.splits((train_ds, valid_ds, test_ds), 
                                        batch_sizes=(batch_size, batch_size, batch_size), 
                                        bptt_len=sequence_length, repeat=False, device=idevice) 

CPU times: user 163 µs, sys: 31 µs, total: 194 µs
Wall time: 201 µs


In [12]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, emb_vectors, nhid, nlayers, dropout=0.5):
        super().__init__()
        ntoken = emb_vectors.shape[0]
        ninp = emb_vectors.shape[1]
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.encoder.weight.data.copy_(emb_vectors);
        self.encoder.weight.requires_grad = False
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [13]:
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    ntokens = len(TEXT.vocab)
    hidden = model.init_hidden(batch_size)
    for i, b in enumerate(data_loader):
        data, targets = b.text, b.target
        output, hidden = model(data)
        output_flat = output.view(-1, ntokens)
        total_loss += len(data) * criterion(output_flat, targets.view(-1)).item()
    return total_loss / len(data_loader)

In [14]:
def train():
    model.train()
    total_loss = 0
    ntokens = len(TEXT.vocab) #len(corpus.dictionary)
    for batch, b in enumerate(train_loader):
        data, targets = b.text, b.target
        model.zero_grad()
        output, hidden = model(data)
        loss = criterion(output.view(-1, ntokens), targets.view(-1))
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), grad_clip)
        for p in filter(lambda p: p.requires_grad, model.parameters()):
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_loader), lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [15]:
def generate(n=50, temp=1.):
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long().to(device)
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
        s = TEXT.vocab.itos[s_idx]
        out.append(s)
    return ' '.join(out)

In [16]:
emb_vectors = TEXT.vocab.vectors
model = RNNModel('LSTM', emb_vectors, 512, 3, 0.3).to(device)
criterion = nn.CrossEntropyLoss()

In [17]:
%%time
with torch.no_grad():
    print('sample:\n', generate(15), '\n')

sample:
 исследованиях познакомился турецкой der обозначена горного сюжета отца. осады написанных 40 горных 21 Начало крепости 

CPU times: user 16.9 ms, sys: 161 µs, total: 17 ms
Wall time: 16.4 ms


In [18]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%time
for epoch in range(1, 11):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')


| epoch   1 | 10000/49916 batches | lr 4.00 | loss  5.03 | ppl   152.49
| epoch   1 | 20000/49916 batches | lr 4.00 | loss  4.49 | ppl    89.01
| epoch   1 | 30000/49916 batches | lr 4.00 | loss  4.28 | ppl    72.16
| epoch   1 | 40000/49916 batches | lr 4.00 | loss  4.15 | ppl    63.41
-----------------------------------------------------------------------------------------
| end of epoch   1 | valid loss 116.98 | valid ppl 635378789003516024374430337499170846140141506920448.00
-----------------------------------------------------------------------------------------
sample:
 произведений , <unk> победой спорта <unk> факультета и снялся <unk> <unk> <unk> фигур <unk> В 1953 г. <unk> <unk> <unk> 2 захватить информации уездного <unk> за свой <unk> Сейчас , белые <unk> <unk> владений <unk> <unk> , <unk> <unk> и семья . <eos> <eos> В январе 2012 года во время 

| epoch   2 | 10000/49916 batches | lr 4.00 | loss  4.00 | ppl    54.40
| epoch   2 | 20000/49916 batches | lr 4.00 | loss  3.94 | 